Import

In [749]:
import pandas as pd

# Load the CSV file
input_file = 's_aes_Quadruplets.csv'  # Replace with your input file name
df = pd.read_csv(input_file)

# Select the first 1000 records
df_first_10000 = df.head(10000)

# Save the selected records to another CSV file
output_file = 'output_first_10000.csv'  # Replace with your desired output file name
df_first_10000.to_csv(output_file, index=False)

print(f"First 1000 records saved to {output_file}")


First 1000 records saved to output_first_10000.csv


In [750]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, SimpleRNN
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
#from keras.layers import SimpleRNN, Dense

Upload the data

In [751]:
data = pd.read_csv('output_first_10000.csv')

Data prepration

In [752]:

# Convert all hexadecimal entries to binary representation in the entire dataframe

y = data[['key']]

x = data.drop(['key','before_xor'], axis=1)
def convert_to_binary(hex_number):
    return bin(int(hex_number, 16))[2:].zfill(16)

x = x.applymap(convert_to_binary)
y = y.applymap(convert_to_binary)





X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=42)

plaintext_train = np.array([list(map(int, list(row))) for row in X_train['plaintext'].values])
ciphertext_train = np.array([list(map(int, list(row))) for row in X_train['ciphertext'].values])
iv_train = np.array([list(map(int, list(row))) for row in X_train['iv'].values])

plaintext_test = np.array([list(map(int, list(row))) for row in X_test['plaintext'].values])
ciphertext_test = np.array([list(map(int, list(row))) for row in X_test['ciphertext'].values])
iv_test = np.array([list(map(int, list(row))) for row in X_test['iv'].values])

# Stack the arrays to create the transformed datasets
X_train_transformed = np.stack([plaintext_train, ciphertext_train, iv_train], axis=-1)
X_test_transformed = np.stack([plaintext_test, ciphertext_test, iv_test], axis=-1)

# For the y_train and y_test
key_train = np.array([list(map(int, list(row))) for row in y_train['key'].values])
key_test = np.array([list(map(int, list(row))) for row in y_test['key'].values])

y_train_transformed = key_train[..., np.newaxis]
y_test_transformed = key_test[..., np.newaxis]
# arr = X_train.apply(lambda row: [[int(row['plaintext'][i]), int(row['ciphertext'][i]), int(row['iv'][i])] for i in range(16)], axis=1).to_numpy()

# X_train_transformed = np.zeros((X_train.shape[0], 16, 3), dtype=int)
# for i in range(X_train.shape[0]):
#     for j in range(16):
#         X_train_transformed[i, j, :] = arr[i][j]

# arr = X_test.apply(lambda row: [[int(row['plaintext'][i]), int(row['ciphertext'][i]), int(row['iv'][i])] for i in range(16)], axis=1).to_numpy()

# X_test_transformed = np.zeros((X_test.shape[0], 16, 3), dtype=int)
# for i in range(X_test.shape[0]):
#     for j in range(16):
#         X_test_transformed[i, j, :] = arr[i][j]

# arr = y_train.apply(lambda row: [[int(row['key'][i])] for i in range(16)], axis=1).to_numpy()

# y_train_transformed = np.zeros((y_train.shape[0], 16,1 ), dtype=int)
# for i in range(y_train.shape[0]):
#     for j in range(16):
#         y_train_transformed[i, j, :] = arr[i][j]



# y_test_transformed = np.zeros((y_test.shape[0], 16,1 ), dtype=int)
# for i in range(y_test.shape[0]):
#     for j in range(16):
#         y_test_transformed[i, j, :] = arr[i][j]







C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13808\1333736298.py:9: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  x = x.applymap(convert_to_binary)
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13808\1333736298.py:10: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  y = y.applymap(convert_to_binary)


In [753]:
y_test.shape

(1000, 1)

Build the model

In [754]:

# Build the SimpleRNN model

# model = Sequential()
# #mlp
# #model.add(Dense(256, activation='relu'))

# model.add(SimpleRNN(16, input_shape=(X_train_transformed.shape[1], X_train_transformed.shape[2])))
# #model.add(Dense(256, activation='relu'))

# model.add(Dense(16, activation='sigmoid'))

# # Compile and train the model
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# model.fit(X_train_transformed, y_train_transformed, epochs=500, batch_size=y_test_transformed.shape[0], verbose=1)


#BidirectionalRNN
model = keras.Sequential()

# Add a Dense layer
model.add(layers.Dense(256, activation='relu'))

# Convert SimpleRNN to Bidirectional RNN
model.add(layers.Bidirectional(layers.SimpleRNN(200), input_shape=(X_train_transformed.shape[1], X_train_transformed.shape[2])))

# Add a Dense layer
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(256, activation='relu'))


# Add a Dense layer with Sigmoid activation
model.add(layers.Dense(16, activation='relu'))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

# Train the model
model.fit(X_train_transformed, y_train_transformed, epochs=200, batch_size=y_test_transformed.shape[0], verbose=1)
model.summary()


c:\Users\Lenovo\Desktop\pandas project\my_venv\Lib\site-packages\keras\src\layers\rnn\bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 4s 130ms/step - accuracy: 0.0284 - loss: 0.3593
Epoch 2/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 118ms/step - accuracy: 0.1208 - loss: 0.2717
Epoch 3/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 127ms/step - accuracy: 0.0246 - loss: 0.2672
Epoch 4/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 129ms/step - accuracy: 0.1261 - loss: 0.2662
Epoch 5/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 124ms/step - accuracy: 0.0618 - loss: 0.2659
Epoch 6/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 124ms/step - accuracy: 0.0318 - loss: 0.2655
Epoch 7/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 125ms/step - accuracy: 0.0894 - loss: 0.2644
Epoch 8/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 126ms/step - accuracy: 0.1051 - loss: 0.2644
Epoch 9/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 138ms/step - accuracy: 0.0750 - loss: 0.2637
Epoch 10/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 126ms/step - accuracy: 0.0725 - loss: 0.2626
Epoch 11/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 140ms/step - accuracy: 0.1534 - loss: 0.2504
Epoch 12/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 121ms/step - accuracy:

Model: "sequential_63"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_193 (Dense)               │ (1000, 16, 256)        │         1,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_56                │ (1000, 400)            │       182,800 │
│ (Bidirectional)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_194 (Dense)               │ (1000, 256)            │       102,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_195 (Dense)               │ (1000, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_196 (Dense)               │ (1000, 16)             │         4,112 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,069,154 (4.08 MB)

 Trainable params: 356,384 (1.36 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 712,770 (2.72 MB)

Evaluate the model

In [755]:

# Evaluate the model

_, accuracy = model.evaluate(X_test_transformed, y_test_transformed)
print(f"Model Evaluation Accuracy: {accuracy}")

# Make predictions
y_pred = (model.predict(X_test_transformed) > 0.5).astype("int32")

# # Calculate accuracy of predictions
# y_test_transformed=y_test_transformed.reshape(y_test_transformed.shape[0],16)
# accuracy = accuracy_score(np.ravel(y_test_transformed), np.ravel(y_pred))
# print(f"Accuracy Score: {accuracy}")
# # accuracy = accuracy_score(y_test_transformed, y_pred)
# # print(f"Accuracy Score: {accuracy}")



32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1082 - loss: 0.0042 
Model Evaluation Accuracy: 0.09299805015325546
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step


In [756]:
y_test_transformed.shape

(1000, 16, 1)

In [757]:
y_test_transformed=y_test_transformed.reshape(y_test_transformed.shape[0],16)
print(y_test_transformed.shape[0])

1000


In [758]:
y_pred.shape

(1000, 16)

In [759]:
y_test_transformed

array([[1, 1, 1, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 1, 1],
       [1, 1, 0, ..., 0, 0, 1],
       ...,
       [0, 1, 1, ..., 0, 0, 0],
       [1, 0, 1, ..., 1, 1, 0],
       [1, 0, 0, ..., 0, 1, 0]])

In [760]:
y_pred

array([[1, 1, 1, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 1, 1],
       [1, 1, 0, ..., 0, 0, 1],
       ...,
       [0, 1, 1, ..., 0, 0, 0],
       [1, 0, 1, ..., 1, 1, 0],
       [1, 0, 0, ..., 0, 1, 0]])

In [761]:
correct_count = 0

# Iterate over each array
for i in range(y_test_transformed.shape[0]):
    if np.array_equal(y_test_transformed[i], y_pred[i]):
        correct_count += 1

# Calculate accuracy
total_arrays = y_test_transformed.shape[0]
accuracy = correct_count / total_arrays

print(f"Number of correct predictions: {correct_count}")
print(f"Accuracy: {accuracy * 100:.2f}%")


Number of correct predictions: 972
Accuracy: 97.20%
